<a href="https://colab.research.google.com/github/AeroEng16/FootballAnalysis/blob/main/SAHIForBallDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAHI

In [2]:
import cv2
vidcap = cv2.VideoCapture('D:/FootballAnalysis/FootballAnalysis/08fd33_4.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("testFrames/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [3]:
from sahi.predict import get_sliced_prediction
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict

# init any model


detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path="bestBallSoFar_0409.pt",
    confidence_threshold=0.3,
    device="cuda:0", # or 'cuda:0'
)
    
predict(
    model_type='yolov8',
    model_path="bestBallSoFar_0409.pt",
    model_device="cuda:0",
    model_confidence_threshold=0.4,
    source="testFrames",
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)


# get sliced prediction result
result = get_sliced_prediction(
    "oldFirmTestImage.png",
    detection_model,
    slice_height = 256,
    slice_width = 256,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)
result.export_visuals(export_dir="demo_data/")

There are 750 listed files in folder: testFrames/


Performing inference on images:   0%|          | 0/750 [00:00<?, ?it/s]

Performing prediction on 60 slices.


Performing inference on images:   0%|          | 1/750 [00:04<58:49,  4.71s/it]

Prediction time is: 4586.41 ms
Performing prediction on 60 slices.


...

Prediction time is: 3353.94 ms
Performing prediction on 60 slices.


Performing inference on images:   3%|▎         | 23/750 [01:25<43:28,  3.59s/it]

Prediction time is: 3437.33 ms


Performing inference on images:   3%|▎         | 23/750 [01:25<43:28,  3.59s/it]

Performing prediction on 60 slices.


In [4]:
from sahi.slicing import slice_coco

coco_dict, coco_path = slice_coco(
    output_coco_annotation_file_name="D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/splitImages/test/labels.json",
    coco_annotation_file_path="D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/test/labels.json",
    output_dir = "D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/splitImages/test/data",
    image_dir="D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/test/data",
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)

ModuleNotFoundError: No module named 'sahi'

# Fifty One Dataset Visualisation

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io
import ast
from plotly.subplots import make_subplots

## Install FiftyOne


## Load a dataset

Let's get started by importing the FiftyOne library:

In [6]:
import fiftyone as fo

In [11]:
import fiftyone.zoo as foz

dataset = fo.Dataset.from_dir(
    dataset_dir = "C:/VSCodeProjects/FootballAnalysis/ballDetect4Coco/splitImages/train",
    dataset_type=fo.types.COCODetectionDataset,
    name="ballDataset2",
)
view = dataset.view()

print(view)

 100% |█████████████| 35604/35604 [13.1s elapsed, 0s remaining, 2.7K samples/s]      


08/27/2024 21:06:23 - INFO - eta.core.utils -    100% |█████████████| 35604/35604 [13.1s elapsed, 0s remaining, 2.7K samples/s]      


Dataset:     ballDataset2
Media type:  image
Num samples: 35604
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    ---


In [ ]:
print(dataset)

Now let's launch the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html) so we can explore the dataset visually. Right away you will see that because we are in a notebook, an embedded instance of the App with our dataset loaded has been rendered in the cell's output.

The [Session](https://voxel51.com/docs/fiftyone/api/fiftyone.core.session.html#fiftyone.core.session.Session) object created below is a bi-directional connection between your Python kernel and the FiftyOne App, as we'll see later.

In [13]:
session = fo.launch_app(dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.25.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



08/27/2024 21:07:00 - INFO - fiftyone.core.session.session -   
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.25.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|

